<a href="https://colab.research.google.com/github/YUHYUNSEONG/YHS/blob/master/bioinfoproject_YHS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2023 생물정보학 실습 프로젝트 유현성

자유 주제 예시에서 mission 3 확장 부분을 진행할 예정입니다. 
2j3와 polyclonal Ab에 대해서 같은 분석을 반복해 볼 계획입니다. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

In [ ]:
!ls -al drive/MyDrive/binfo*

In [ ]:
!conda install -y subread

In [ ]:
%cd /content/drive/MyDrive/binfo1-work